## 02 Data Cleaning and Preprocessing
**Objetivo**: Limpiar y preprocesar los datos para que estén listos para el modelado.
**Contenido**:
- Manejo de valores nulos.
- Conversión de tipos de datos.
- Codificación de variables categóricas.
- Normalización y estandarización de las variables.
- Creación de nuevas características (feature engineering).
